In [1]:
import pandas as pd 
import numpy as np

In [2]:
data1 = pd.read_csv("MSFT_AAPL_Log_Returns (1).csv")
data1.head()

,Date,MSFT Price,APPL Price,MSFT Log Price,AAPL Log Price,MSFT Log Return,AAPL Log Return
0,31-Aug-11,26.60,54.98,3.280911,4.006969,NaN,NaN
1,1-Sep-11,26.21,54.43,3.266141,3.996915,-0.014770,-0.010054
2,2-Sep-11,25.80,53.44,3.250374,3.978560,-0.015767,-0.018356
3,6-Sep-11,25.51,54.25,3.239071,3.993603,-0.011304,0.015043
4,7-Sep-11,26.00,54.85,3.258097,4.004602,0.019026,0.010999


In [3]:
data = data1.drop(data1.index[0])

In [4]:
ValPort = 1000000
MarketCapMSFT = 448.77
MarketCapAAPL = 577.1
WeightMSFT = MarketCapMSFT/ (MarketCapMSFT+MarketCapAAPL)
WeightAAPL = MarketCapAAPL/(MarketCapMSFT+MarketCapAAPL)

Full Loss Operator and Linear Loss Operator 

Full Loss Operator is: 
$l_{[t]}(x) = -V_t ( w_t^M (e^{x^m} -1) + w_t^A (e^{x^A} -1))$

Linear Loss Operator is: 
$l'_{[t]}(x) = -V_t ( w_t^M x^m + w_t^A x^A)$

where $w_t^M$ , $w_t^A$ are the weights and $x^m$, $x^A$ are the one day log returns for Apple and Microsoft

In [5]:
FullLoss = -ValPort * ((WeightAAPL*(np.exp(data["AAPL Log Return"])-1))
                      +(WeightMSFT*(np.exp(data["MSFT Log Return"])-1)))
    
LinearLoss = -ValPort * ((WeightAAPL*data["AAPL Log Return"])
                       +(WeightMSFT*data["MSFT Log Return"]))

In [6]:
def ValueAtRisk (data_column, alpha = 0.95):
    "This function computes the value at risk for a dataframe"
    "It should take in a Series of data "
    sorted_df = data_column.sort_values(ascending = True)
    VaR = sorted_df.quantile(q= alpha, interpolation = 'higher')
    return VaR

In [7]:
def ExpectedShortFall (data_column, alpha = 0.95):
    sorted_df = data_column.sort_values(ascending = True)
    VaR = ValueAtRisk(sorted_df, alpha = alpha )
    Multipler = 1/(len(sorted_df)*(1-alpha))
    FirstTerm = VaR
    SecondTerm = (sorted_df.searchsorted(VaR)[0]+1) - len(sorted_df)*alpha 
    #SecondTerm = 1
    ThirdTerm = sum(sorted_df[sorted_df > VaR])
    ES = Multipler *(FirstTerm * SecondTerm + ThirdTerm)
    return ES

In [8]:
print(" Full Loss VaR: ", ValueAtRisk(FullLoss), '\n',
      "Full Loss Expected Shortfall: ", ExpectedShortFall(FullLoss), '\n',     
     "Linear Loss VaR:  ", ValueAtRisk(LinearLoss), '\n',
      "Linear Loss Expected Shortfall: ", ExpectedShortFall(LinearLoss)
     )

 Full Loss VaR:  19566.5452585 
 Full Loss Expected Shortfall:  29348.7045053 
 Linear Loss VaR:   19834.3583327 
 Linear Loss Expected Shortfall:  30011.6378995
